VCN.TO.csv is what you get when you click Download Data [here](https://finance.yahoo.com/quote/VCN.TO/history?period1=1483938000&period2=1572580800&interval=1d&filter=history&frequency=1d) It doesn't bring down the dividend data so I manually copied it from the link above into dividends.csv. For vcn_monthly_total_returns.csv I manually copied the 1-year NAV values from [here](https://www.vanguardcanada.ca/individual/indv/en/product.html#/fundDetail/etf/portId=9561/assetCode=equity/?performance). I honestly wasn't sure on NAV vs Market price but NAV lined up best with Yahoo's 1 year Total Monthly Total Returns under their Performance tab. Of note, some months Vanguard and Yahoo line up exactly (ex: 1-Year Monthly Total Returns for Oct 2019: 12.13 on both) but some months they don't quite (ex: 1-Year Monthly Total Returns for Sept 2019: 6.15 on Vanguard vs 6.18 on Yahoo). I don't have a good sense of what would explain these differences. I would welcome any insight. Summary thoughts are at the end.

In [1]:
import pandas as pd

In [2]:
# bring in the data
returns = pd.read_csv('vcn_monthly_total_returns.csv')
values = pd.read_csv('VCN.TO.csv')
dividends = pd.read_csv('dividends.csv')

In [3]:
# make three data frames to consider 3 possible calculation options
# closing price, adjusted closing price, closing price and dividends
returns_adjusted = returns.copy(deep=True)
returns_base = returns.copy(deep=True)
returns_base_divs = returns.copy(deep=True)
for time_type in ('same-first-last', 'same-last-last', 'next-first-last'):
    returns_adjusted[time_type] = 0
    returns_base[time_type] = 0
    returns_base_divs[time_type] = 0

In [4]:
# given 2019-10; expect: 2018-10-01 to 2019-10-31
def same_first_last(returns, values, column, dividends=False):
    """Version 1 of calculating 1 year total monthly returns.
       In this case, given an ending period of 2019-10, would consider:
       2018-10-01 to 2019-10-31
    
    Args:
        returns(df): the returns df we want to add the results to
        values(df): df with daily stock data
        column(str): column in values to use for calculations
        dividends: pass in df of dividends if you want to consider them as well
    
    Returns:
        taking advantage of df mutability means no need to return anything
    """
    for month in returns.Month.tolist():
        start_month = str(int(month[:4]) - 1)+'-'+month[5:]
        max_date = values.loc[values.Date.apply(lambda x: x[0:7])==month, 'Date'].max()
        max_date_val = values.loc[values.Date == max_date, column].iloc[0]
        min_date = values.loc[values.Date.apply(lambda x: x[0:7])==start_month, 'Date'].min()
        min_date_val = values.loc[values.Date == min_date, column].iloc[0]
        if dividends is not False:
            divs = dividends.loc[(dividends.Date>=min_date) & (dividends.Date<=max_date),'Amount'].sum()
            returns.loc[returns['Month']==month, 'same-first-last'] = (max_date_val / (min_date_val - divs) - 1) * 100
        else:
            returns.loc[returns['Month']==month, 'same-first-last'] = (max_date_val / min_date_val -1) * 100

In [5]:
# given 2019-10; expected: 2018-10-31 to 2019-10-31
def same_last_last(returns, values, column, dividends=False):
    """Version 1 of calculating 1 year total monthly returns.
       In this case, given an ending period of 2019-10, would consider:
       2018-10-31 to 2019-10-31
    
    Args:
        returns(df): the returns df we want to add the results to
        values(df): df with daily stock data
        column(str): column in values to use for calculations
        dividends: pass in df of dividends if you want to consider them as well
    
    Returns:
        taking advantage of df mutability means no need to return anything
    """
    for month in returns.Month.tolist():
        start_month = str(int(month[:4]) - 1)+'-'+month[5:]
        max_date = values.loc[values.Date.apply(lambda x: x[0:7])==month, 'Date'].max()
        max_date_val = values.loc[values.Date == max_date, column].iloc[0]
        min_date = values.loc[values.Date.apply(lambda x: x[0:7])==start_month, 'Date'].max()
        min_date_val = values.loc[values.Date == min_date, column].iloc[0]
        if dividends is not False:
            divs = dividends.loc[(dividends.Date>=min_date) & (dividends.Date<=max_date),'Amount'].sum()
            returns.loc[returns['Month']==month, 'same-last-last'] = (max_date_val / (min_date_val - divs) - 1) * 100
        else:
            returns.loc[returns['Month']==month, 'same-last-last'] = (max_date_val / min_date_val - 1) * 100

In [6]:
# given 2019-10; expected: 2018-11-01 to 2019-10-31
def next_first_last(returns, values, column, dividends=False):
    """Version 1 of calculating 1 year total monthly returns.
       In this case, given an ending period of 2019-10, would consider:
       2018-11-01 to 2019-10-31
    
    Args:
        returns(df): the returns df we want to add the results to
        values(df): df with daily stock data
        column(str): column in values to use for calculations
        dividends: pass in df of dividends if you want to consider them as well
    
    Returns:
        taking advantage of df mutability means no need to return anything
    """
    for month in returns.Month.tolist():
        start_month = str(int(month[:4]) - 1)+'-'+month[5:]
        max_date = values.loc[values.Date.apply(lambda x: x[0:7])==month, 'Date'].max()
        max_date_val = values.loc[values.Date == max_date, column].iloc[0]
        min_date = values.loc[values.Date.apply(lambda x: x[0:7])>start_month, 'Date'].min()
        min_date_val = values.loc[values.Date == min_date, column].iloc[0]
        if dividends is not False:
            divs = dividends.loc[(dividends.Date>=min_date) & (dividends.Date<=max_date),'Amount'].sum()
            returns.loc[returns['Month']==month, 'next-first-last'] = (max_date_val / (min_date_val - divs) - 1) * 100
        else:
            returns.loc[returns['Month']==month, 'next-first-last'] = (max_date_val / min_date_val - 1) * 100

In [7]:
for fct in (same_first_last, same_last_last, next_first_last):
    fct(returns_base, values, 'Close')
    fct(returns_adjusted, values, 'Adj Close')
    fct(returns_base_divs, values, 'Close', dividends)

In [8]:
# first tried percent error but does not perform well for near 0 values
# for time_type in ('same-first-last', 'same-last-last', 'next-first-last'):
#     for df in (returns_base, returns_adjusted, returns_base_divs):
#     df[time_type+'_err'] = (df[time_type]-df['Monthly_Return'])/df['Monthly_Return']

In [9]:
# calculate absolute difference and round
for time_type in ('same-first-last', 'same-last-last', 'next-first-last'):
    for df in (returns_base, returns_adjusted, returns_base_divs):
        df[time_type+'_err'] = round((df[time_type] - df['Monthly_Return']),1)

In [10]:
# printing out returns_base, see below for commentary
returns_base

,Month,Monthly_Return,same-first-last,same-last-last,next-first-last,same-first-last_err,same-last-last_err,next-first-last_err
0,2019-10,12.13,1.775335,8.873605,8.130081,-10.4,-3.3,-4.0
1,2019-09,6.15,1.943517,3.101962,2.754827,-4.2,-3.0,-3.4
2,2019-08,3.31,-0.210084,0.392515,0.971758,-3.5,-2.9,-2.3
3,2019-07,2.05,0.454824,-0.897395,-0.570225,-1.6,-2.9,-2.6
4,2019-06,2.93,1.040710,-0.181439,0.090958,-1.9,-3.1,-2.8
5,2019-05,1.95,2.269149,-0.734163,-0.673392,0.3,-2.7,-2.6
6,2019-04,8.83,9.064324,5.899050,5.798928,0.2,-2.9,-3.0
7,2019-03,7.32,4.138592,4.406558,5.458083,-3.2,-2.9,-1.9
8,2019-02,6.45,0.964221,3.392257,4.138592,-5.5,-3.1,-2.3
9,2019-01,0.10,-4.162865,-2.533993,-1.897359,-4.3,-2.6,-2.0


Here we only considered change in closing stock price. As we can see above, it did not perform well over any of the time intervals considered. Don't consider this surprising as I expected dividends to be included.

In [11]:
# printing out returns_adjusted, see below for commentary
returns_adjusted

,Month,Monthly_Return,same-first-last,same-last-last,next-first-last,same-first-last_err,same-last-last_err,next-first-last_err
0,2019-10,12.13,4.821925,12.132674,11.366900,-7.3,0.0,-0.8
1,2019-09,6.15,5.684115,6.188264,5.830738,-0.5,0.0,-0.3
2,2019-08,3.31,2.718608,3.338902,3.935148,-0.6,0.0,0.6
3,2019-07,2.05,3.403011,2.011119,2.347888,1.4,-0.0,0.3
4,2019-06,2.93,4.616869,2.748095,3.028469,1.7,-0.2,0.1
5,2019-05,1.95,5.057032,1.971844,2.034278,3.1,0.0,0.1
6,2019-04,8.83,12.037460,8.785890,8.683036,3.2,-0.0,-0.1
7,2019-03,7.32,7.677855,7.252705,8.332902,0.4,-0.1,1.0
8,2019-02,6.45,3.635382,6.127645,6.893730,-2.8,-0.3,0.4
9,2019-01,0.10,-1.627349,0.044625,0.698099,-1.7,-0.1,0.6


Here we considered the change in adjusted closing price. This performed quite well which makes sense as adjusted closing price should include the effect of dividends. The middle time period (Oct 31, 2018 - Oct 31, 2019) is the most accurate. This makes sense as it lines up with what Vanguard uses on their 1 year graphs. Not sure it logically makes sense as (Nov 1, 2018 - Oct 31, 2019) makes more sense to me. Also not sure why there isn't 0 error. It's low enough that I feel compfortable that I know what's going on though. If anyone does know why error isn't 0, please do reach out.

In [12]:
returns_base_divs

,Month,Monthly_Return,same-first-last,same-last-last,next-first-last,same-first-last_err,same-last-last_err,next-first-last_err
0,2019-10,12.13,4.808890,12.352295,11.560672,-7.3,0.2,-0.6
1,2019-09,6.15,5.667685,6.185752,5.817577,-0.5,0.0,-0.3
2,2019-08,3.31,2.626624,3.264080,3.877035,-0.7,-0.0,0.6
3,2019-07,2.05,3.340719,1.910243,2.256246,1.3,-0.1,0.2
4,2019-06,2.93,4.610997,2.678149,2.966399,1.7,-0.3,0.0
5,2019-05,1.95,5.101218,1.931842,1.995923,3.2,-0.0,0.0
6,2019-04,8.83,12.180447,8.834492,8.728745,3.4,0.0,-0.1
7,2019-03,7.32,7.790393,7.359018,8.471171,0.5,0.0,1.2
8,2019-02,6.45,3.596837,6.154749,6.941650,-2.9,-0.3,0.5
9,2019-01,0.10,-1.723114,-0.009511,0.660642,-1.8,-0.1,0.6


Here we considered the change in closing price and added the effect of dividends. This performed quite well which makes sense. The middle time period (Oct 31, 2018 - Oct 31, 2019) again performs best (aligning with the above). Again, not sure why there isn't 0 error. It's low enough that I feel compfortable that I know what's going on though. If anyone does know why error isn't 0, please do reach out.